# [Fluent Python](https://www.amazon.com/Fluent-Python-Concise-Effective-Programming/dp/1491946008)
## Chapter 7: Function Decorators and Closures

In [1]:
def decorate(func):
    def inner():
        print('running inner()')
    return inner

@decorate
def target():
    print('running target()')

In [2]:
target()

running inner()


In [4]:
registry = []

def register(func):
    print('running register(%s)' % func)
    registry.append(func)
    return func

@register
def f1():
    print('running f1()')
    
@register
def f2():
    print('running f2()')
    
def f3():
    print('running f3()')


running register(<function f1 at 0x7f4901279400>)
running register(<function f2 at 0x7f4901279730>)


In [5]:
registry

[<function __main__.f1()>, <function __main__.f2()>]

In [6]:
f1()

running f1()


In [9]:
f2()

running f2()


In [10]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price
        
    def total(self):
        return self.price * self.quantity
    
class Order:
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion
        
    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total
    
    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion(self)
        return self.total() - discount
    
    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}'
        return fmt.format(self.total(), self.due())

In [8]:
promos = []

def promotion(promo_func):
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity(order):
    """5% discount for customers with 1000 or more fidelity points"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0

@promotion
def bulk_item(order):
    """10% discount for each LineItem with 20 or more units"""
    discount = 0
    for item in order.cart:
        if item.quantity >= 20:
            discount += item.total() * 0.1
    return discount

@promotion
def large_order(order):
    """7% discount for orders with 10 or more distinct items"""
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10:
        return order.total() * 0.07
    return 0

def best_promo(order):
    """Select best discount available"""
    return max(promo(order) for promo in promos)

In [12]:
import time

def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', .'.join(repr(arg) for arg in args)
        print('[%0.8fs] %s(%s) -> %r' % (elapsed, name, arg_str, result))
        return result
    return clocked

In [13]:
@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)

In [16]:
factorial(10)

[0.00000075s] factorial(1) -> 1
[0.00010750s] factorial(2) -> 2
[0.00015998s] factorial(3) -> 6
[0.00035720s] factorial(4) -> 24
[0.00044665s] factorial(5) -> 120
[0.00049648s] factorial(6) -> 720
[0.00053700s] factorial(7) -> 5040
[0.00058070s] factorial(8) -> 40320
[0.00061829s] factorial(9) -> 362880
[0.00066208s] factorial(10) -> 3628800


3628800

In [17]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [19]:
fibonacci(10)

[0.00000051s] fibonacci(0) -> 0
[0.00000059s] fibonacci(1) -> 1
[0.00008490s] fibonacci(2) -> 1
[0.00000042s] fibonacci(1) -> 1
[0.00000046s] fibonacci(0) -> 0
[0.00000048s] fibonacci(1) -> 1
[0.00003804s] fibonacci(2) -> 1
[0.00007780s] fibonacci(3) -> 2
[0.00021526s] fibonacci(4) -> 3
[0.00000022s] fibonacci(1) -> 1
[0.00000032s] fibonacci(0) -> 0
[0.00000049s] fibonacci(1) -> 1
[0.00004050s] fibonacci(2) -> 1
[0.00007915s] fibonacci(3) -> 2
[0.00000021s] fibonacci(0) -> 0
[0.00000022s] fibonacci(1) -> 1
[0.00002345s] fibonacci(2) -> 1
[0.00000020s] fibonacci(1) -> 1
[0.00000025s] fibonacci(0) -> 0
[0.00000020s] fibonacci(1) -> 1
[0.00002337s] fibonacci(2) -> 1
[0.00004564s] fibonacci(3) -> 2
[0.00009150s] fibonacci(4) -> 3
[0.00028992s] fibonacci(5) -> 5
[0.00054664s] fibonacci(6) -> 8
[0.00000047s] fibonacci(1) -> 1
[0.00000031s] fibonacci(0) -> 0
[0.00000034s] fibonacci(1) -> 1
[0.00002813s] fibonacci(2) -> 1
[0.00005630s] fibonacci(3) -> 2
[0.00000034s] fibonacci(0) -> 0
[0.00000

55

In [20]:
import functools

@functools.lru_cache()
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)

In [22]:
fibonacci(30)

[0.00000182s] fibonacci(11) -> 89
[0.00011844s] fibonacci(12) -> 144
[0.00000263s] fibonacci(13) -> 233
[0.00022623s] fibonacci(14) -> 377
[0.00000217s] fibonacci(15) -> 610
[0.00031859s] fibonacci(16) -> 987
[0.00000220s] fibonacci(17) -> 1597
[0.00040460s] fibonacci(18) -> 2584
[0.00000217s] fibonacci(19) -> 4181
[0.00048944s] fibonacci(20) -> 6765
[0.00000153s] fibonacci(21) -> 10946
[0.00057735s] fibonacci(22) -> 17711
[0.00000206s] fibonacci(23) -> 28657
[0.00065695s] fibonacci(24) -> 46368
[0.00000212s] fibonacci(25) -> 75025
[0.00074866s] fibonacci(26) -> 121393
[0.00000221s] fibonacci(27) -> 196418
[0.00084111s] fibonacci(28) -> 317811
[0.00000214s] fibonacci(29) -> 514229
[0.00093371s] fibonacci(30) -> 832040


832040